In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df=pd.read_csv("train.csv")

/home/lawrence/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (12,13,14,18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Splitting the calldate

In [3]:
from datetime import datetime
df.calldate=pd.to_datetime(df.calldate)

In [4]:
df["newcalldate"]=df["calldate"].apply(lambda x: datetime.strftime(x,"%Y%m%d%H%M%S"))

In [5]:
df["year"]=df["calldate"].apply(lambda x: datetime.strftime(x,"%Y"))

In [6]:
df["month"]=df["calldate"].apply(lambda x: datetime.strftime(x,"%m"))

In [7]:
df["day"]=df["calldate"].apply(lambda x: datetime.strftime(x,"%d"))

In [8]:
df["hour"]=df["calldate"].apply(lambda x: datetime.strftime(x,"%H"))

In [9]:
df["minute"]=df["calldate"].apply(lambda x: datetime.strftime(x,"%M"))

In [10]:
df["sec"]=df["calldate"].apply(lambda x: datetime.strftime(x,"%S"))

In [11]:
df.drop(["calldate","newcalldate","year","minute","sec"],axis=1,inplace=True)

In [12]:
df.head(3)

,cc_status,maincat,subcat1,casepriority,referal,caller_gender,caller_age,caller_county,child_age,child_gender,...,parent_age,parent_gender,parent_county,Abuser_Relationship,Neglector_Relationship,Perpetrator_Relationship,Physical abuser_Relationship,month,day,hour
0,Closed,non-interventional,Blank call,Non Critical,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,01,01,07
1,Closed,non-interventional,Blank call,Non Critical,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,01,01,07
2,Closed,non-interventional,Blank call,Non Critical,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,01,01,07


## Counting Number Of Calls Per Hour Per Day

In [13]:
df["num_calls"]=1

In [14]:
data= (df.groupby(['month',"day", 'hour'])['num_calls'].sum().reset_index())

In [15]:
data.head(3)

,month,day,hour,num_calls
0,01,01,07,8
1,01,01,08,41
2,01,01,09,20


### Inserting KenyaPublicHolidays2016 and NairobiSchoolDates2016 from the csv_files provided by Zindi

In [16]:
pub_holiday=pd.Series([])
for i in range(len(data)):
    if data["month"][i]=="01" and data["day"][i]=="01":
        pub_holiday[i]="holiday"
    elif data["month"][i]=="03" and data["day"][i]=="25":
        pub_holiday[i]="holiday"
    elif data["month"][i]=="03" and data["day"][i]=="27":
        pub_holiday[i]="holiday"
    elif data["month"][i]=="05" and data["day"][i]=="02":
        pub_holiday[i]="holiday"
    
    elif data["month"][i]=="06" and data["day"][i]=="01":
        pub_holiday[i]="holiday"
    elif data["month"][i]=="06" and data["day"][i]=="07":
        pub_holiday[i]="holiday"
    elif data["month"][i]=="09" and data["day"][i]=="07":
        pub_holiday[i]="holiday"
    else :
        pub_holiday[i]="nopublicholiday"

data.insert(4,"pubHolidays",pub_holiday)
        
        
        

In [17]:
schdate=pd.Series([])
for i in range(len(data)):
    if data["month"][i]=="02" or data["month"][i]=="03":
        schdate[i]="schooltime"
    elif data["month"][i]=="01" and (data["day"][i]=="01"):
        schdate[i]="notschooltime"
    elif data["month"][i]=="01" and (data["day"][i]=="02"):
        schdate[i]="notschooltime"
    elif data["month"][i]=="01" and (data["day"][i]=="03"):
        schdate[i]="notschooltime"
    
    elif data["month"][i]=="04" and data["day"][i]=="01" :
        schdate[i]="schooltime"
    elif data["month"][i]=="04" and (data["day"][i]=="02"):
        schdate[i]="schooltime"
    elif data["month"][i]=="04" and (data["day"][i]=="03"):
        schdate[i]="schooltime"
    elif data["month"][i]=="04" and (data["day"][i]=="04"):
        schdate[i]="schooltime"
    elif data["month"][i]=="04" and (data["day"][i]=="05"):
        schdate[i]="schooltime"
    elif data["month"][i]=="04" and (data["day"][i]=="06"):
        schdate[i]="schooltime"
    elif data["month"][i]=="04" and (data["day"][i]=="07"):
        schdate[i]="schooltime"
    elif data["month"][i]=="04" and (data["day"][i]=="08"):
        schdate[i]="schooltime"
    elif data["month"][i]=="04":
        schdate[i]="notschooltime"
    elif data["month"][i]=="06" or data["month"][i]=="07":
        schdate[i]="schooltime"
    elif data["month"][i]=="05" and data["day"][i]=="01":
        schdate[i]="notschooltime"
    elif data["month"][i]=="05" and data["day"][i]!="01":
        schdate[i]="schooltime"
    else:
        schdate[i]="schooltime"
data.insert(5,"schooltimes",schdate)

In [18]:
data["year"]="2016"

In [19]:
data.head()

,month,day,hour,num_calls,pubHolidays,schooltimes,year
0,01,01,07,8,holiday,notschooltime,2016
1,01,01,08,41,holiday,notschooltime,2016
2,01,01,09,20,holiday,notschooltime,2016
3,01,01,10,34,holiday,notschooltime,2016
4,01,01,11,44,holiday,notschooltime,2016


In [20]:

data['time_index'] = data[['year','month',"day","hour"]].apply(lambda x: '-'.join(x), axis=1)

In [21]:
data.head()

,month,day,hour,num_calls,pubHolidays,schooltimes,year,time_index
0,01,01,07,8,holiday,notschooltime,2016,2016-01-01-07
1,01,01,08,41,holiday,notschooltime,2016,2016-01-01-08
2,01,01,09,20,holiday,notschooltime,2016,2016-01-01-09
3,01,01,10,34,holiday,notschooltime,2016,2016-01-01-10
4,01,01,11,44,holiday,notschooltime,2016,2016-01-01-11


In [22]:
data.to_csv("ts.csv",index=False)#this is the training dataset

## Generating the dates that we'll Forcast on its num_calls

In [23]:
date_rng = pd.date_range(start='13/07/2016', end='16/09/2016', freq='H')

In [24]:
df2 = pd.DataFrame(date_rng, columns=['date'])

In [25]:
df2.shape

(1561, 1)

In [26]:
df2.head(2)

,date
0,2016-07-13 00:00:00
1,2016-07-13 01:00:00


In [27]:
final_test=df2.iloc[0:1344]

In [28]:
final_test.shape

(1344, 1)

In [29]:
final_test.head(3)

,date
0,2016-07-13 00:00:00
1,2016-07-13 01:00:00
2,2016-07-13 02:00:00


In [30]:
final_test.date=pd.to_datetime(final_test.date)

/home/lawrence/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [31]:
final_test["year"]=final_test["date"].apply(lambda x: datetime.strftime(x,"%Y"))

/home/lawrence/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [32]:
final_test["month"]=final_test["date"].apply(lambda x: datetime.strftime(x,"%m"))

/home/lawrence/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [33]:
final_test["day"]=final_test["date"].apply(lambda x: datetime.strftime(x,"%d"))

/home/lawrence/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [34]:
final_test["hour"]=final_test["date"].apply(lambda x: datetime.strftime(x,"%H"))

/home/lawrence/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [35]:
final_test.head()

,date,year,month,day,hour
0,2016-07-13 00:00:00,2016,07,13,00
1,2016-07-13 01:00:00,2016,07,13,01
2,2016-07-13 02:00:00,2016,07,13,02
3,2016-07-13 03:00:00,2016,07,13,03
4,2016-07-13 04:00:00,2016,07,13,04


### Inserting KenyaPublicHolidays2016 and NairobiSchoolDates2016 from the csv_files provided by Zindi

In [36]:
pub_holiday=pd.Series([])
for i in range(len(final_test)):
    
    if final_test["month"][i]=="09" and final_test["day"][i]=="07":
        pub_holiday[i]="holiday"
    else :
        pub_holiday[i]="nopublicholiday"

final_test.insert(4,"pubHolidays",pub_holiday)
        
        
        

In [37]:
schdate=pd.Series([])
for i in range(len(final_test)):
    if final_test["month"][i]=="07":
        schdate[i]="schooltime"
   
    if final_test["month"][i]=="08" and final_test["day"][i]=="01" :
        schdate[i]="schooltime"
    elif final_test["month"][i]=="08" and final_test["day"][i]=="02":
        schdate[i]="schooltime"
    elif final_test["month"][i]=="08" and final_test["day"][i]=="03":
        schdate[i]="schooltime"
    elif final_test["month"][i]=="08" and final_test["day"][i]=="04":
        schdate[i]="schooltime"
    elif final_test["month"][i]=="08" and final_test["day"][i]=="05":
        schdate[i]="schooltime"
    elif final_test["month"][i]=="08":
        schdate[i]="notschooltime"
    
    
    elif final_test["month"][i]=="09" and final_test["day"][i]=="01":
        schdate[i]="notschooltime"
    elif final_test["month"][i]=="09" and final_test["day"][i]=="02":
        schdate[i]="notschooltime"
    elif final_test["month"][i]=="09" and final_test["day"][i]=="03":
        schdate[i]="notschooltime"
    elif final_test["month"][i]=="09" and final_test["day"][i]=="04":
        schdate[i]="notschooltime"
        
    elif final_test["month"][i]=="09":
        schdate[i]="schooltime"
    else:
        schdate[i]="schooltime"
final_test.insert(6,"schooltimes",schdate)

In [38]:
final_test['time_index'] = final_test[['year','month',"day","hour"]].apply(lambda x: ''.join(x), axis=1)

/home/lawrence/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [39]:
final_test.head()

,date,year,month,day,pubHolidays,hour,schooltimes,time_index
0,2016-07-13 00:00:00,2016,07,13,nopublicholiday,00,schooltime,2016071300
1,2016-07-13 01:00:00,2016,07,13,nopublicholiday,01,schooltime,2016071301
2,2016-07-13 02:00:00,2016,07,13,nopublicholiday,02,schooltime,2016071302
3,2016-07-13 03:00:00,2016,07,13,nopublicholiday,03,schooltime,2016071303
4,2016-07-13 04:00:00,2016,07,13,nopublicholiday,04,schooltime,2016071304


In [40]:
final_test.to_csv("tsTest.csv",index=False)#this is the test dataset

In [41]:
testdataset=final_test["time_index"]#extracting the time_index from the test dataset

In [42]:
testdataset.to_csv("submission.csv",index=False)#holds the time_index column


/home/lawrence/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.
